In [1]:
# === Import libraries ===
import os
import pandas as pd
import numpy as np
import base64        # Untuk mengubah gambar ke format base64 (teks)
import requests      # Untuk mengirim permintaan HTTP ke LMStudio
import Levenshtein   # Untuk menghitung edit distance dan CER
import editdistance  # Untuk menghitung edit distance dan CER

In [2]:
# === Class Names ===
# Membaca daftar label dari file classes.names
class_names_file = r"C:\Users\adity\Downloads\Indonesian License Plate Recognition Dataset\classes.names"

# Membaca setiap baris sebagai nama kelas (huruf/angka)
with open(class_names_file, "r") as f:
    class_names = [line.strip() for line in f.readlines()]

print("Jumlah kelas:", len(class_names))   # Menampilkan jumlah karakter
print("Contoh class:", class_names)        # Menampilkan isi dari class_names

Jumlah kelas: 36
Contoh class: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [3]:
# === Konfigurasi koneksi ke LMStudio ===
LMSTUDIO_API_URL = "http://localhost:1234/v1/chat/completions"   # URL endpoint lokal LMStudio
MODEL_NAME = "qwen/qwen2.5-vl-7b"                                # Nama model multimodal yang digunakan

In [4]:
# === Fungsi untuk membaca ground truth dari file label YOLO ===
def read_ground_truth(label_path, class_names_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()  # Membaca semua baris label

    with open(class_names_path, 'r') as f:
        class_names = f.read().splitlines()     # Membaca daftar nama kelas

    entries = []
    for line in lines:
        parts = line.strip().split()            # Memisahkan setiap elemen (class, x_center, dst.)
        cls_idx = int(parts[0])                 # Mengambil indeks kelas (misalnya 0 = 'A')
        x_center = float(parts[1])              # Mengambil posisi tengah objek pada sumbu X
        entries.append((x_center, class_names[cls_idx]))  # Simpan posisi dan label karakter

    entries.sort(key=lambda x: x[0])            # Urutkan berdasarkan posisi X (kiri ke kanan)
    gt = ''.join([char for _, char in entries]) # Gabungkan semua karakter jadi 1 string
    return gt         

In [5]:
# === Fungsi untuk mengubah gambar menjadi base64 ===
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')  # Encode ke base64 lalu decode ke string teks

In [6]:
# === Fungsi untuk melakukan prediksi ke model VLM via API ===
def get_prediction(image_path):
    base64_image = encode_image_to_base64(image_path)  # Ubah gambar ke base64
    headers = {"Content-Type": "application/json"}     # Header HTTP untuk format JSON
    data = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are an AI model that extracts license plate numbers from vehicle images."},
            {"role": "user", "content": [
                {"type": "text", "text": "What is the license plate number shown in this image? Respond only with the plate number."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]}
        ],
        "temperature": 0  # Gunakan prediksi deterministik (tidak random)
    }

    response = requests.post(LMSTUDIO_API_URL, headers=headers, json=data)  # Kirim permintaan ke API
    response.raise_for_status()  # Munculkan error jika gagal
    result = response.json()     # Ambil hasil prediksi dalam bentuk JSON
    return result['choices'][0]['message']['content'].strip()  # Ambil hanya hasil teks prediksi

In [7]:
# === Fungsi untuk menghitung CER (Character Error Rate) ===
def calculate_cer(gt, pred):
    return round(editdistance.eval(gt, pred) / max(len(gt), 1), 4)
    # CER = jumlah karakter berbeda / panjang GT
    # Gunakan max(..., 1) agar tidak terjadi pembagian dengan nol

In [8]:
# === Main Execution (bagian utama program) ===
image_dir = r"C:\Users\adity\Downloads\Indonesian License Plate Recognition Dataset\images\test"   # Direktori gambar
label_dir = r"C:\Users\adity\Downloads\Indonesian License Plate Recognition Dataset\labels\test"   # Direktori label
classes_path = r"C:\Users\adity\Downloads\Indonesian License Plate Recognition Dataset\classes.names"

results = []  # Menyimpan hasil prediksi untuk semua gambar

# Loop semua file gambar dalam folder
for img_name in os.listdir(image_dir):
    if not img_name.endswith(".jpg"):
        continue  # Lewati file yang bukan gambar .jpg
    
    img_path = os.path.join(image_dir, img_name)  # Path lengkap ke gambar
    label_name = img_name.replace(".jpg", ".txt") # Nama file label yang sesuai
    label_path = os.path.join(label_dir, label_name)

    try:
        gt_text = read_ground_truth(label_path, classes_path)  # Baca ground truth dari label
        pred_text = get_prediction(img_path)                   # Dapatkan prediksi dari model
        cer = calculate_cer(gt_text, pred_text)                # Hitung CER
    except Exception as e:
        pred_text = "ERROR"                                    # Jika error saat prediksi
        gt_text = read_ground_truth(label_path, classes_path)  # Tetap ambil GT
        cer = 1.0                                              # Anggap kesalahan total
        print(f"[!] Gagal memproses {img_name}: {e}")

    results.append([img_name, gt_text, pred_text, cer])        # Simpan hasil ke list

    print(f"🖼️  {img_name} | GT: {gt_text} | Pred: {pred_text} | CER: {cer}")

🖼️  test001_1.jpg | GT: B9140BCD | Pred: B S140 BCD | CER: 0.375
🖼️  test001_2.jpg | GT: B2407UZO | Pred: B2407UZ0 | CER: 0.125
🖼️  test001_3.jpg | GT: B2842PKM | Pred: B2842PKM | CER: 0.0
🖼️  test002_1.jpg | GT: BG1352AE | Pred: BG1352AE | CER: 0.0
🖼️  test003_1.jpg | GT: B2634UZF | Pred: B2634 UZF | CER: 0.125
🖼️  test003_2.jpg | GT: B1995JVK | Pred: B1995JVKG128 | CER: 0.5
🖼️  test004_1.jpg | GT: B9062VEH | Pred: 89062 YEH | CER: 0.375
🖼️  test005_1.jpg | GT: DD8798KM | Pred: DD798KM | CER: 0.125
🖼️  test006_1.jpg | GT: T1329KC | Pred: T-1329-KC | CER: 0.2857
🖼️  test007_1.jpg | GT: AD8865EE | Pred: AD8865EE | CER: 0.0
🖼️  test008_1.jpg | GT: DK1157AAB | Pred: DK1157AAB | CER: 0.0
🖼️  test008_2.jpg | GT: AA1997FE | Pred: AA-1997-FE | CER: 0.25
🖼️  test009_1.jpg | GT: H8518NA | Pred: H *8518* NA | CER: 0.5714
🖼️  test009_2.jpg | GT: K1649GB | Pred: K1649GB | CER: 0.0
🖼️  test010_1.jpg | GT: B9416PCN | Pred: B9416PCN | CER: 0.0
🖼️  test010_2.jpg | GT: H1147UY | Pred: H1147UY | CER: 0.

In [9]:
# === Simpan hasil evaluasi ke file CSV ===
df = pd.DataFrame(results, columns=["image", "ground_truth", "prediction", "CER_score"])
df.to_csv("results.csv", index=False)   # Simpan ke file CSV tanpa index
print("✅ Hasil disimpan ke results.csv")

✅ Hasil disimpan ke results.csv


In [10]:
# === Evaluasi statistik keseluruhan: Akurasi dan CER total ===
total_S = total_D = total_I = total_N = 0  # Inisialisasi variabel untuk kesalahan
benar = salah = 0                          # Hitung jumlah benar/salah

for _, gt, pred, _ in results:
    if pred == gt:
        benar += 1
    else:
        salah += 1

    # Hitung jenis kesalahan dengan Levenshtein
    ops = Levenshtein.editops(gt, pred)
    S = sum(1 for op in ops if op[0] == 'replace')  # Substitusi
    D = sum(1 for op in ops if op[0] == 'delete')   # Penghapusan
    I = sum(1 for op in ops if op[0] == 'insert')   # Penambahan
    N = len(gt)                                     # Jumlah karakter ground truth

    total_S += S
    total_D += D
    total_I += I
    total_N += N

In [11]:
# Hitung CER total dan akurasi akhir
overall_CER = round((total_S + total_D + total_I) / max(total_N, 1), 4)
accuracy = round(benar / (benar + salah) * 100, 2)

# === Cetak Nilai SDIN (Substitusi, Deletion, Insertion, N) ===
print("\n========= Detail SDIN =========")
print(f"Substitusi (S) : {total_S}")
print(f"Deletion   (D) : {total_D}")
print(f"Insertion  (I) : {total_I}")
print(f"Total Char (N) : {total_N}")
print("==================================")

# === Tampilkan ringkasan hasil evaluasi ke layar ===
print("\n========= Statistik Evaluasi =========")
print(f"Jumlah prediksi BENAR: {benar}")
print(f"Jumlah prediksi SALAH: {salah}")
print(f"Persentase Akurasi   : {accuracy}%")
print(f"Total CER (S+D+I)/N  : {overall_CER}")
print("========================================")


========= Detail SDIN =========
Substitusi (S) : 41
Deletion   (D) : 14
Insertion  (I) : 52
Total Char (N) : 1455

========= Statistik Evaluasi =========
Jumlah prediksi BENAR: 136
Jumlah prediksi SALAH: 61
Persentase Akurasi   : 69.04%
Total CER (S+D+I)/N  : 0.0735
